# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**8/05/2020 - C9 S6**

# Computación de alto Rendimiento con Python

Python es utilizado transversalmente, ya sea en la industria o en la academia. Dentro de sus cualidades se encuentra la portabilidad de código, sintaxis intuitiva, disponibilidad de herramientas y documentación. Sin embargo, al ser un lenguaje interpretado se pierden ciertas características intrínsicas de los lenguajes de bajo nivel como C, C++ y Fortran.

Se estudian distintas herramientas para mejorar el rendimiento del interprete CPython, estas se basan en el uso eficiente de objetos base, aplicación de técnicas de paralelismo y compilación utilizando tanto librerías nativas, como desarrolladas por terceros. 

## Perfilamiento y Referenciación

El flujo de trabajo en ciencia de datos consta de numerosas rutinas de carga, procesamiento, visualización y optimización. Estas rutinas son abordadas por medio de técnicas de programación y diseño de código. En este apartado, se debe tener en cuenta la importancia de generar rutinas eficientes, pues significan reducciones en los tiempos de respuesta y uso de recursos. Por lo anterior, es natural que desde fases tempranas del desarrollo de proyectos, se busque optimizar el código. Como directriz general, se recomienda llevar el proceso de desarrollo en dos etapas. La primera consiste en generar rutinas **correctas**, **comprensibles** y **mantenibles**, evitando la sobre-optimización prematura de código. Como segunda etapa, se recomienda comenzar con los procesos de optimización de rutinas. Esto pues, las herramientas que permiten mejorar los aspectos computacionales, interfieren en la sencillez del código, entorpeciendo los procesos de depuración y mantención. 

Una vez que las rutinas están implementadas de manera correcta, la mejor manera de enfocar los esfuerzos, pasa por **perfilar** (*profiling*) las rutinas codificadas. Esto consiste en encontrar las zonas de código criticas en cuanto a carga computacional. La manera más directa de encontrar estas zonas, es por medio del uso de contadores de tiempo o *timers*.

**Ejemplo**

Se utiliza la librería `time` para medir el tiempo de ejecución de una zona de código.

In [ ]:
from math import sin, cos
import time

Se define la función a analizar

In [ ]:
def func_1(a):
    
    result = 0
    for val in a:
        result += sin(val) + cos(val)**2
    return result

Se define un rango de datos a operar y se estudia el tiempo de ejecución por medio de la función `process_time`.

In [ ]:
x = [0.1 * i for i in range(1000)]

t0 = time.process_time()
for r in range(1000):
    func_1(x)
t1 = time.process_time()


print("Tiempo transcurrido", t1 - t0)

**Ejercicio**

1. Defina una clase *context manager* llamada `Timer`. Esta debe abstraer la dinámica de medición temporal anterior. *Hint*: Deberá definir los métodos `__enter__` y `__exit__`, en este último se produce el `print` de tiempo transcurrido.

En algunas ocasiones se desea medir el tiempo de ejecución para tareas sencillas, la librería estándar de Python provee el módulo `timeit`, este puede ser utilizado directamente en la consola interactiva IPython o en notebooks de Jupyter por medio del comando mágico `%timeit`. 

**Ejemplo**

Se mide el tiempo de ejecución promedio para la función coseno de NumPy.

In [ ]:
import numpy as np

In [ ]:
%timeit np.cos(0.5)

Se compara con la función del módulo `Math`

In [ ]:
%timeit cos(0.5)

Se puede ver una gran diferencia en los tiempos de ejecución promedio. 

**Ejercicio**

1. Diferencie los comandos `%time`, `%timeit`, `%%timeit`.

Un **perfilador**  (*profiler*) es un programa que ejecuta una rutina y monitorea las funciones ahí especificadas, obteniendo métricas de rendimiento como el consumo de tiempo y memoria. Por otra parte, la referenciación (*benchmarking*) consiste en extraer zonas de código de interés para probar su rendimiento antes y después de aplicar técnicas de optimización. IPython provee de un perfilador de código dado por la orden `%prun`.

**Ejemplo**

1. Se perfila una función utilizando `%prun`. En primera instancia se define tal función

In [ ]:
def benchmark_sum(n):
    '''Funcion de referencia que suma n elementos transformados
    '''
    ac = 0
    
    for i in range(n):
        to_sum = [(i // 2)**n + (i-n)**(n // 3) for i in range(n)]
        ac = sum(to_sum)
    
    return ac

Se perfila la función con `%prun`

In [ ]:
%prun benchmark_sum(500)

EL resultado corresponde las mediciones temporales de cada función involucrada en la ejecución de `benchmark_sum(500)`. En este caso, la mayoria del tiempo se utiliza en la ejecución de la compresión de listas `<listcomp>`. Esto indica que la mejor manera de optimizar el código de `benchmark_sum`pasa por optimizar tal sección del código.

Con `%lprun` es posible perfilar por linea de código, para ello es necesario instalar el módulo `line_profiler`. 

**Ejemplo**

Se carga la extensión `%lprun` y se prueba con `benchmark_sum`.

In [ ]:
%load_ext line_profiler

El comando `%lprun` toma como parámetro una orden de Python y su principal argumento. Las funciones que se desean perfilar deben ser especificadas de manera explicita con la orden `-f`. En el caso de `benchmark_sum` esto se haría según el siguiente código

In [ ]:
%lprun -f benchmark_sum benchmark_sum(500)

El resultado es una tabla con el tiempo utilizado en cada linea de las funciones perfiladas, mostrando porcentajes del tiempo consumido en cada paso.

**Ejercicio**

1. Los perfiladores `%prun` y `%lprun` tienen en común los argumentos -D, -T y -r utilice el parámetro `?` para investigar estas opciones.

Es posible perfilar uso de memoria, para ello existen los comandos mágicos `%memit` y `%mprun`. Para utilizarlos es necesario instalar el módulo `memory_profiler` y cargarlo mediante

```python
%load_ext memory_profiler
```

**Ejemplo**

Se carga la extensión y se perfila el uso de memoria para `benchmark_sum`

In [ ]:
%load_ext memory_profiler

En primera instancia perfilamos utilizando la linea mágica `%memit`, la cual es equivalente a `timeit` pero ofrece medidas sobre el uso de memoria. 

In [ ]:
%memit benchmark_sum(500)

Se puede observar un uso de memoria en torno a 90 MB.

De manera análoga a `%prun` la librería `memory_profiler` permite utilizar `%mprun`, con la cual se pueden obtener descripciones linea a linea del uso de memoria. El uso de este comando es un poco más restrictivo, pues solo permite medir funciones definidas en módulos (no dentro de un notebook). Para ello, se crea el módulo `memory_demo`. La manera sencilla de hacer esto, es mediante el comando mágico `%%file` este permite crear archivos en el directorio de trabajo actual, utilizando el código dentro de una celda de jupyter. 

Se procede a generar el módulo que contiene el código de `benchmark_sum` utilizando el comando `%%file`.




In [ ]:
%%file bench_module.py

def benchmark_sum(n):
    '''Funcion de referencia que suma n elementos transformados
    '''
    ac = 0
    
    for i in range(n):
        to_sum = [(i // 2)**n + (i-n)**(n // 3) for i in range(n)]
        ac = sum(to_sum)
    
    return ac

a continuación, se importa el módulo creado y se perfuila su memoria mediante:

In [ ]:
from bench_module import benchmark_sum
%mprun -f benchmark_sum benchmark_sum(500)

El resultado se muestra en pantalla, obteniendo detalles linea a linea.

**Ejercicio**

1. Agregue la linea `del to_sum` luego de ejecutar `ac = sum(to_sum)` y antes de terminar el ciclo `for` principal. Estudie el efecto en el consumo de memoria en la función.

## Optimización con Código Nativo

Una de las manera más eficientes de mejorar el rendimiento de aplicaciones es por medio del uso de algoritmos más eficientes en conjunción de estructuras de datos mejor diseñadas. A continuación, se estudian los algoritmos y estructuras de datos presentes de manera nativa en Python que permiten acelerar ciertas rutinas. 

En términos generales, lso algoritmos pueden ser clasificados según su *complejidad computacional*, esta clasificación se expresa según la notación de O-grande, que corresponde a una cota superior de las operaciones requeridas para ejecutar una tarea.

Si la tarea no depende del tamaño del input (acceder a cierta llave de un diccionario por ejemplo) se dice que el algoritmo asociado se efectúa en tiempo constante, denotado por $O(1)$. Esto quiere decir, que sin importar la cantidad de datos disponibles, el tiempo de ejecución de la tarea será siempre el mismo. 

**Ejemplo**

Se genera un lista, a cada uno de sus elementos se le realiza una operación básica.

In [ ]:
lista = list(range(10))

for i in range(len(lista)):
    lista[i]+=100

En este algoritmo, la operación `lista[i]+=100` es repetida tantas veces como elementos hay en `lista`, que corresponde al tamaño de los datos de entrada. Al observar la que las operaciones realizadas por este algoritmo son proporcionales a la cantidad de elementos de `lista`, se puede decir que su tiempo de ejecución es $O(N)$ donde `N = len(lista)`. 

### Optimización de Operaciones con Listas

Las listas de Python son colecciones ordenadas de elementos, estás se encuentran clasificadas como *arreglos*, que a la vez corresponden a una estructura de datos caracterizada por contener elementos contiguos en bloques de memoria, cada uno de los cuales contienen una referencia a un objeto de Python. La ventaja de las listas recae en la facilidad que entregan par acceder, modificar y agregar elementos. Dado que acceder y modificar elementos de una lista corresponde a acceder a espacios de memoria que a priori no dependen de la longitud de la lista, se dice que estas operaciones tienen complejidad $O(1)$. Por otra parte, para agregar un elemento a una lista por medio de `.append()`, puede requerirse re-ubicar la memoria del arreglo asociado, operación que toma un tiempo de $O(N)$. Sin embargo, tal operación es muy poco frecuente, pues por lo general se tiene acceso a bloques de memoria contiguos, por tal motivo, se dice que la operación `.append()` tiene un tiempo esperado de ejecución de $O(1)$. 

Para agregar o eliminar datos al inicio de un arreglo, se requiere hacer una traslación (o *shift*) de los demás elementos por lo que tal operación toma un tiempo de $O(N)$. Para agregar o remover elementos de un arreglo en una posición distinta a la última, se opera de manera análoga. 

**Ejemplo**

Se definen listas para estudiar la complejidad de ciertos métodos empíricamente. Se definen los parámetros

In [ ]:
n_0, n_1, n_2 = (int(10e5), int(5*10e5), int(10e6))

Se generan una funciones de referencia

In [ ]:
def copy_objs(obj_0,obj_1,obj_2):
    '''Abstraccion auxiliar para copiar elementos.'''
    return (obj_0.copy(),obj_1.copy(),obj_2.copy())

def bench_pop(l_0,l_1,l_2, index = -1):
    '''Funcion de referncia para eliminacion de elementos.'''
    
    l_0.pop(index)
    l_1.pop(index)
    l_2.pop(index)
    

def bench_append(l_0,l_1,l_2, index = 1):
    '''Funcion de referencia para insertar 1 con append.'''
    
    l_0.append(index)
    l_1.append(index)
    l_2.append(index)
    

def bench_insert(l_0,l_1,l_2, index = (0,1)):
    '''Funcion de referncia para insertar 1 con insert.'''
    
    l_0.insert(*index)
    l_1.insert(*index)
    l_2.insert(*index)

Se construye el test 

In [ ]:
lista_0, lista_1, lista_2 = (list(range(n_0)), list(range(n_1)),
                             list(range(n_2)))

Se elimina el ultimo elemento

In [ ]:
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo constante
%lprun -f bench_pop bench_pop(l_0,l_1,l_2)

Se elimina el primer elemento

Se elimina el primer elemento

In [ ]:
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo lineal
%lprun -f bench_pop bench_pop(l_0,l_1,l_2,0)

Se inserta 1 en la ultima posicion

In [ ]:
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo constante (casi seguramente)
%lprun -f  bench_append bench_append(l_0,l_1,l_2)

Se inserta 1 en la primera posicion

In [ ]:
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo lineal
%lprun -f bench_insert bench_insert(l_0,l_1,l_2)

Para efectuar inserciones de manera eficiente (siempre en tiempo constante) Se puede utilizar la estructura de datos `deque` del módulo `collections`. Estas estructuras se comportan como listas, están diseñadas para acelerar la inserción de objetos y añaden los métodos `.popleft` y `.appendleft`. 

**Ejemplo**

Se compara `.popleft` en *deques* con `.pop(0)` en listas.

In [ ]:
from collections import deque

def bench_pop_left(d_0,d_1,d_2):
    '''Funcion de referncia para eliminacion de elementos.'''
    
    d_0.popleft()
    d_1.popleft()
    d_2.popleft()
    
def bench_append_left(d_0,d_1,d_2, val = 1):
    '''Funcion de referncia para insertar 1 con insert.'''

    d_0.appendleft(val)
    d_1.appendleft(val)
    d_2.appendleft(val)

Se definen los objetos sobre los que se trabajará

In [ ]:
deque_0, deque_1, deque_2 = tuple(map(deque, [lista_1, lista_1, lista_2]))

In [ ]:
d_0, d_1, d_2 = copy_objs(deque_0,deque_1,deque_2)

# Se observa un tiempo constante
%lprun -f bench_pop_left bench_pop_left(d_0, d_1, d_2)

In [ ]:
%%timeit 
d_0, d_1, d_2 = copy_objs(deque_0,deque_1,deque_2)

Copiar objetos de tipo `deque` tiene una carga de aproximadamente 190 ms

In [ ]:
%%timeit 
d_0, d_1, d_2 = copy_objs(deque_0,deque_1,deque_2)
bench_pop_left(d_0, d_1, d_2)

Por su parte, aplicar el benchmark `bench_pop_left` tarda en promedio 191 - 190 ms = 1 ms.

En cuanto a las listas, la operación de copiar lleva unos 112 ms en promedio

In [ ]:
%%timeit 
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)

Aplicar el benchmark `bench_pop` en listas lleva un tiempo promedio de 120 ms - 112 ms = 8ms. Por lo que se aprecia un aumento en el rendimiento. Cabe señalar que tal aumento se ve sujeto a una carga mayor en el proceso de copia de objetos, por tal motivo, vale la pena evitar la copia de objetos tipo `deque` y utilizarlos para acceder a lista con una gran cantidad de elementos.

In [ ]:
%%timeit 
l_0,l_1,l_2 = copy_objs(lista_0, lista_1, lista_2)
bench_pop(lista_0,lista_1,lista_2,0)

**Ejercicios**

1. Repita el proceso de comparación para `.appendleft` en deques  e `.insert()` en listas. 

2. Cual es la complejidad computacional de acceder a:
    1. Primer elemento de un deque / lista
    2. Ultimo elemento de un deque / lista
    3. Un elemento distinto del último o el primero (ej: el elemento de la mitad de un arreglo).

3. El módulo `bisect` permite hacer búsquedas rápidas en arreglos ordenados. la función `bisect.bisect` permite encontrar el índice en cual insertar un elemento, manteniendo el orden del arreglo operado. 
    1. Genere la lista ordenada 'ordered_list'  de números entre 0 y 10.
    2. Elimine el cuarto elemento de la lista, guarde su valor en la variable `dropped`. Se debe hacer en una linea de código. 
    3. Importe el módulo `bisect` y utilice el comando `bisect.bisect(ordered_list,dropped)`. ¿Qué significa el valor retornado por la función?¿qué operación se efectúa por medio del comando recién aplicado?
    4. Haga un código de referencia para comparar las funciones `list.index()` y `bisect.bisect()` por medio de perfilamiento temporal. Para comprobar sus resultados utilice el hecho de que  el tiempo de ejecución para `bisect.bisect` es de $O(\log(N)$, mientras que el de `list.index()` es de $O(N)$. 
    5. Estudie la función `bisect.bisect_left`. 
    

### Optimización de Operaciones con Diccionarios

La gran flexibilidad de los diccionarios los hacen un objeto central en el uso de Python. Estos son implementaciones de *hash maps*, es decir, son estructuras de datos construidas por medio de asociaciones *llave - valor*, donde a cada llave, se asigna un índice especifico, de tal manera que el valor de tal índice puede ser ordenado en un arreglo. Por tal motivo, los diccionarios son altamente eficientes en procesos de eliminación, acceso e inserción teniendo un tiempo promedio de ejecución de $O(1)$. 

Para acceder a los índices dados por el *hash map* se puede utilizar la función `hash` de Python, esta opera sobre distintos tipos de datos.

**Ejemplo**

Se aplica `hash` a diferentes objetos

In [ ]:
print('hash string: ',hash('MA6202'))
print('hash int:' , hash(1234))
print('hash tuple', hash(('a','b','c')))

Un objeto puede ser operado por `hash` (*hashable object*) si tiene un método `__hash__` y puede ser comparado por medio de `__eq__` por ejemplo. Si un objeto es *hashable* significa que puede ser utilizado como llave de un diccionario, en general, todos los objetos inmutables de Python son *hashables* mientras que las listas y diccionarios, por ser inmutables, no lo son.  

**Ejemplo**

Es posible usar las ventajas de accesibilidad de diccionarios agrupar listas de manera eficiente. Para esto se utiliza el objeto `defaultdict` de la librería `collections`

In [ ]:
from collections import defaultdict

Se construye la lista a agrupar

In [ ]:
to_group = [('a', 1), ('b', 2), ('c', 3), ('b', 4), ('d', 1)]

Los objetos `defaultdict` son subclases de `dict`. Reciben como argumentos un valor inicial para su el atributo `.default_factory`, cuyo valor por defecto es `None`. 

Los objetos `defaultdict` poseen todas las funcionalidades de un diccionario pero añaden el método `.__missing__()` con el cual se proveen valores por defecto, los cuales se asignan a una nueva llave, es decir, permiten inicializar diccionarios entregando solo el valor de la llave (y no su valor asociado), pues a cada llave nueva, se asigna un valor por defecto de manera automática. 

Según lo enterior, inicializar un objeto `defaultdict` por medio de `defaultdict(list)` genera un diccionario, en el cual, cada llave nueva tendrá asociada una lista vacía (valor por defecto del atributo `.default_factory` para este tipo de dato).

In [ ]:
D = defaultdict(list)

Una vez teniendo el diccionario definido, es posible agrupar los elementos de la lista `to_group` y se perfila por medio de:

In [ ]:
%%timeit
D = defaultdict(list)

for k, v in to_group:
    D[k].append(v)

sorted(D.items())

El código anterior genera las llaves `k`, que por defecto poseen una lista vacía asociada, a cada lista vacía agregan por medio de `append` (tiempo de ejecución constante) el elemento inspeccionado `v`. 

Se implementa la misma funcionalidad usando ciclos `for` y `append` de listas, se perfila utilizando `%%timeit`

In [ ]:
%%timeit
L = []
for elem in to_group:    
    if len(L) == 0:
        L.append((elem[0],[elem[1]]))
    else:
        c = 0
        for l in L:
            if l[0] == elem[0]:
                l[1].append(elem[1])
                c = 1
        if c == 0:
             L.append((elem[0],[elem[1]]))

L

Con lo anterior apreciamos una ganancia en eficiencia, que tambien se traduce en simpleza.

**Ejercicios**

Los objetos `defaultdict` permiten además aumentar la eficiencia al momento de contar elementos de un arreglo. Para ver esto, implemente una función que:

1. Reciba una *iterable* como argumento.
2. Inicalice un objeto `defaultdict` con tipo de dato `int`. ¿Qué valor se asocia por defecto?.
3. Recorra cada elemento del iterable, registrando su número de ocurrencias en una llave del objeto `defaultdict` antes inicializado.
    

El módulo `collections` permite implementar el procedimiento del ejercicio 2 anterior por medio de la clase `Counter`

**Ejemplo**

Se cuentan los elementos de una lista por medio de la clase `Counter`

In [ ]:
from collections import Counter
import numpy as np

lista = np.random.randint(0, 10, size=100)
counts = Counter(lista)

sorted(counts.items())

**Obs**: El método de counteo por medio de la clase `Counter` tiene un tiempo de ejecución de $O(N)$.

Otra ventaja de los diccionarios es que permiten buscar palabras de manera rápida en una lista de documentos, en este caso, tal lista de documentos viene representada dada por:

In [ ]:
with open('text.txt','r') as file:
    lines = file.readlines()
    lines = [l.rstrip(' \n') for l in lines]
        
lines

supongamos que se busca la palabra 'imaginario' en cada documento, es posible generar una lista de documentos con tal palabra por medio de:

In [ ]:
to_search = 'imaginario'
%timeit found = [line for line in lines if to_search in line]

Se debe considerar que el tiempo de ejecución asociado a consultar por una palabra es $O(N)$. Para mejorar esto, se puede construir un diccionario, donde a cada palabra se asocie un índice, donde este último corresponde al la linea (o documento si se prefiere) al que pertenece. Esto se puede hacer mediante el siguiente código

In [ ]:
index = defaultdict(list)

for i, line in enumerate(lines):
    
    for word in line.split():
        index[word].append(i)

En el diccionario generado, hacer búsquedas es de orden $O(1)$, luego para la misma consulta antes hecha se tiene

In [ ]:
%%timeit
res = index[to_search]
[lines[i] for i in res]

Es decir, un aumento substancial de rendimiento. Cabe mencionar, que este procedimiento solo tiene sentido si se busca hacer una cantidad alta de consultas sobre un arreglo de lineas/documentos, esto pues, el tiempo de preprocesamiento para generar el indexado por diccionario puede ser muy alto.

### Optimización de Operaciones con Conjuntos

A diferencia de las listas, los conjuntos son colecciones no ordenadas, donde cada elemento debe ser único. La implementación de conjuntos en Python sigue la misma lógica de los diccionarios en cuanto ambos utilizan funciones *hash*. Por tal motivo, en conjuntos se tienen operaciones rápidas para añadir, eliminar y acceder a elementos. (Del orden $O(1)$).

**Ejercicio**

Los tiempos de ejecución para los métodos `A.union(B)`, `A.intersection(B)` y `A.difference(B)` son $O(a+b)$, $O(\min(a,b))$ y $O(a)$, donde $a = |A|$ y $b = |B|$. 

1. Construya una función de referencia para cada método y utilice un perfilamiento adecuado para comprobar la afirmación anterior de manera empírica. 

Se puede hacer uso de conjuntos para efectuar consultas rápidas.

**Ejemplo**

Se utiliza el objeto `index` creado para hacer búsquedas sobre texto. Se consulta sobre los documentos donde las palabras 'imaginario' y 'vive' ocurren simultáneamente, se obtiene un estimado del tiempo de ejecución.

In [ ]:
to_search = ['imaginaria', 'vive']

In [ ]:
%%timeit

res_0 = [lines[i] for i in index[to_search[0]]]
res_1 = [lines[i] for i in index[to_search[1]] ]

[r for r in res_1 if r in res_0]

Se puede modificar la función de indexación para que opere sobre conjuntos.

In [ ]:
index_set = defaultdict(set)

for i, line in enumerate(lines):
    
    for word in line.split():
        index_set[word].add(i)

luego se hace la misma búsqueda por medio de

In [ ]:
%%timeit
index_set[to_search[0]].intersection(index_set[to_search[1]])

En este apartado se ve un incremento substancial.

### Optimización con uso de Memoización

También se puede mejorar el rendimiento de aplicaciones por medio de un uso eficiente de la memoria, una de las ideas tras esta premisa es la de guardar los resultados de operaciones intensivas en un espacio de memoria llamado *cache*, este espacio puede estar ubicado en memoria (RAM), disco o almacenada de manera remota. El acto de guardar resultados en memoria para luego utilizarlos de manera directa se denomina **memoización**. y es una forma de *chaching* o uso de memorias *cache*.

Python ofrece el decorador `@lru_cache` accesible desde la librería base `functools`. Este decorador puede ser utilizado de manera sencilla para guardar resultados en memoria y luego accederlos. 

**Ejemplo**

Se utiliza el decorador `@lru_cache` sobre una función sencilla. En primera instancia se importa el módulo necesario.

In [ ]:
from functools import lru_cache

Se define la función a memoizar y se aplica el decorador

In [ ]:
@lru_cache
def simple_func(x, y):
    '''Funcion de prueba para memoizar.'''

    print('Obteniendo el resultado...')

    return x**y + y**x

Para comprobar el funcionamiento del decorador se llama la función dos veces sobre el mismo argumento.

In [ ]:
args = (2,5)
simple_func(*args)

Se repite el procedimiento 

In [ ]:
simple_func(*args)

En este último caso se observa que el resultado es obtenido directamente desde la memoria. 

**Ejercicios**

`@lru_cache` es un decorador que acepta argumentos de entrada, permitiendo el uso del `max_size`. Con este parámetro se especifica el tamaño máximo de memoria para el cache asociado a la función. 

1. Decore la función anterior indicando como parámetro `max_size = 8`.

2. ¿Qué ocurre cuando se llena el tamaño maximo y se realizan más cálculos? *Hint*: lru significa *least recently used*.

3. Acceda a la información del *cache* por medio del método `.cache_info()` de la función decorada. ¿Qué significa *hit* y *miss* en este contexto?


Un ejemplo más avanzado es el de memoizar funciones recursivas

**Ejemplo**

Se trabaja con la función `factorial` almacenando sus resultados en *cache*.

In [ ]:
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

Se mide el tiempo de ejecución para `n=1000`.

In [ ]:
%%timeit
factorial(1000)

Se memoiza la función y se repite el experimento

In [ ]:
@lru_cache
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

In [ ]:
%%timeit
factorial(1000)

Con lo que se comprueba la eficiencia del método. 

**Ejercicios**

1. Programe secuencia de Fibonacci, perfile su consumo de tiempo y luego compare con una versión memoizada.

2. Instale el módulo `joblib`. Este módulo permite guardar resultados en disco por medio del objeto `Memory`. Utilice el decorador `@memory.cache` para memoizar as funciones anteriores (factorial y fibonacci). Compare los tiempos de ejecución al guardar los resultados en disco vs ram.

**Obs**: La ventaja de utilizar técnicas de *caching* tienen un costo, este radica en aumentar el consumo de memoria, si esta memoria esta localizada en disco, el acceso puede ser muy lento y el rendimiento puede decaer drásticamente. Antes de usar este tipo de estrategias, se recomienda estudiar la factibilidad, teniendo en cuenta las politicas de almacenamiento y acceso de los resultados y su relación con el rendimiento del programa que se desea implementar.

### Optimización con uso de Compresiones y Generadores

Las compresiones de lista están altamente optimizadas en Python y por tanto puede ser utilizadas para reemplazar ciclos `for` en ciertas circunstancias. 

**Ejemplo**

Es posible ganar un mayor rendimiento al utilizar comprensión de listas en vez del ciclo `for` en el siguiente código

In [ ]:
n = int(10e4)

def for_loop(n=n):
    res = []
    for i in range(n):
        res.append(i*(i+1) - i**2)
    
    return sum(res)/n

Se mide su consumo de tiempo por medio de `%%timeit`

In [ ]:
%%timeit
for_loop()

Se implementa la misma función haciendo uso de compresión de listas y de diccionarios.

In [ ]:
def list_comp(n=n):
    return sum([i*(i+1) - i**2 for i in range(n)])/n
    
def dic_comp(n=n):
    return sum({i: i*(i+1) -i**2 for i in range(n)})/n

In [ ]:
%%timeit
list_comp()

In [ ]:
%%timeit
dic_comp()

En el caso de compresión de listas vemos una mejoría, por su parte, en compresión de diccionarios, podemos esperar que operaciones de reducción sean más lentas pues se hace uso de llaves. 

En términos de memoria, cada compresión de listas (o diccionarios) ocupa un nuevo espacio, lo cual aumenta el uso de memoria. Para atacar este problema, se puede hacer uso de **generadores**. 

Un generador es un iterable que guarda que posee memoria solo de su estado actual y una regla de cambio para el estado siguiente. 

**Ejemplo**

La función `map` toma como argumentos una función y un iterator, el resultado de su aplicación es un generador. Para estudiar el comportamiento de este tipo de objetos se construyen dos funciones y se perfilan ...



In [ ]:
# %%file bench_module.py -a

def list_comp_list(n=int(10e6)):
    '''Concatena operaciones sobre comprensiones de lista'''

    l_1 = [i**2 for i in range(n) if i % 2 == 0]
    l_2 = [i * (i - 1) for i in l_1]

    l_3 = [i // 3 for i in l_2]

    return max(l_3) / n**2

se construye la misma función utilizando generadores `map`

In [ ]:
# %%file bench_module.py -a

def list_comp_map(n=int(10e6)):
    '''Concatena operaciones sobre comprensiones de lista'''

    l_1 = map(lambda i: i**2, [i for i in range(n) if i % 2 == 0])
    l_2 = map(lambda i: i * (i - 1), l_1)
    
    l_3 = map(lambda i: i // 3, l_2)

    return max(l_3) / n**2

se perfila la memoria utilizada por estas funciones

In [ ]:
%memit list_comp_list()

In [ ]:
%memit list_comp_map()

**Ejercicios**

Se pueden entender los objetos generadores con las estructuras ya vistas. El objetivo de estos ejercicios es revisar ciertos aspectos de interés al usar generadores.

1. Se puede definir un generador por medio de *compresnsión de generadores* este tipo de comprensión sigue la misma sintaxis que una compresión usua (listas o diccionarios) solo que se utilizan paréntesis normales (como una '*compresion de tuplas*'). Defina un generador por medio de *compresión de generadores*.

2. Se pueden definir generadores a partir de funciones, en este caso, el comando `return` debe ser sustituido por `yield`. Este comando permite que el generador definido sea un iterable, el cual solo guarda su estado actual de ejecución y posee una *regla de transición* dada por el cuerpo de la función. Implemente un generador de secuencias inifinitas. Para ello:
    1. Defina una función `infinite_seq`, que no tiene argumentos de entrada. 
    2. Defina un acumulador de suma por medio de la variable `sum = 0`.
    3. Defina un bloque `while True` dentro de este inserte las ordenes `yield sum` y en la linea siguiente `sum +=1`. 
    4. Itere sobre su generador utilizando el método `next`. Interprete la función de `yield` en el bloque anterior. 

## Uso eficiente de Arreglos con Numpy y Pandas

Como ya se ha discutido, NumPy provee una rutinas altamente eficientes para realizar operaciones matemáticas complejas basándose en arreglos de C y FORTRAN. Esto permite tener velocidades optimas aún cuando Python sea interpretado. Otra de las cualidades de NumPy es que almacena resultados intermedios en memoria, es posible mejorar tal aspecto por medio del paquete `numexpr` , el cual permite optimizar y compilar arreglos de manera rápida.

**Ejemplo**

se definen arreglos de NumPy

In [ ]:
import numpy as np

In [ ]:
n = int(10e7)
x, y, z = (np.random.rand(n), np.random.rand(n), np.random.rand(n))

Para obtener las ventajas de `numexpr` es recomendable trabajar con arreglos de gran tamaño. A continiación se utiliza la función `numexpr.evaluate` para procesar una operación entre los arreglos definidos, esta función actúa como `eval` de Python, por lo que recibe un string y lo ejecuta.

In [ ]:
from numexpr import evaluate

In [ ]:
%%timeit
evaluate('x + z*y**2')

In [ ]:
%%timeit 
x + z*y**2

**Ejercicio**

1. Estudie la documentación de `numexpr` y busque las funciones soportadas por `evaluate`.

2. Genere una matriz de distancia entre dos arreglos de NumPy. Esta contiene la distancia euclidiana en cada una de sus componentes. Compare los tiempos de ejecución de obtener tal matriz con solamente con expresiones de NumPy y utilizando `numexpr`. 

Como consideraciones generales al trabajar con NumPy se puede tener en cuenta:

1. Las operaciones *inplace* son más rápidas que sus contrpartes.

se define una operacion *inplace* y se compara con una asignación

In [ ]:
n = int(5*10e5)

In [ ]:
%%timeit
a = np.random.rand(n)
a *= 5

In [ ]:
%%timeit
a = np.random.rand(n)
b = a * 5

2. Aplicar *reshape* no implica generar una copia, por su parte, trasponer si. Por lo anterior, trasponer utiliza más memoria que cambiar la forma de un arreglo. Vale la pen

3. `flatten()` y `ravel()` permiten cambiar la forma de un arreglo a dimensión 1, sin embarog, `flatten()` retorna una copia, mientras que `ravel()` solo lo hace si es necesario. Por tal motivo `ravel()` es más rápido con arreglos de gran tamaño.

In [ ]:
n = int(100)

In [ ]:
%%timeit
a = np.random.random(size = [n,n])
b = a.flatten()

In [ ]:
%%timeit
a = np.random.random(size = [n,n])
b = a.ravel()

4. Seimrpe mejor utilizar [reglas de broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html) para operar con arreglos de distinto tamaño, es decir, se debe prevenir el uso de `reshape` si es que dos arreglos son compatibles según alguna regla de *broadcasting*. A modo de ejemplo, se estudia el producto externo entre dos arreglos.

In [ ]:
n = int(1000)

In [ ]:
a = np.random.random(size = n)
a_1 = a[:,np.newaxis]
a_2 = a[np.newaxis, :]

'''
Obs: np.tile permite copiar un arreglo segun un patron (rep_f,rep_c)
de esta forma se repite tantas veces por fila como rep_f y tantas
veces por columna como rep_c.
'''

%timeit np.tile(a_1, (1, n)) * np.tile(a_2, (n, 1))

In [ ]:
%timeit a_1 * a_2

5. En arreglos de gran tamaño, donde la precisión no es un problema, se puede reducir el consumo de memoria al disminuir la presición numérica de los elementos del arreglo.

In [ ]:
a.dtype

In [ ]:
b = a.astype('float16')
b.dtype

**Ejercicio**

Perfile el uso de memoria y velocidad de ejecución para arreglos de punto flotante de doble presición  `float64`versus punto flotante de presición singular `float32`.

## Compiladores

## Paralelismo

## Procesamiento Distribuido